In [3]:
print("12")

12


In [2]:
import os
import time
# import numpy
# import argparse
from os.path import join, exists
import glob
import cv2
import numpy as np
# from train import VideoCoverageLoss
# import torch
from operator import itemgetter

def MAP(target, indices, count_frames, k=10):

    #assert logits.shape == target.shape

    #target = target.reshape(-1,k)
   # logits = logits.reshape(-1,k)
    
    #sorted, indices = numpy.sort(logits, 1)[::-1], numpy.argsort(-logits, 1)
    count_nonzero = 0
    map_sum = 0
    for i in range(len(indices)):
        average_precision = 0
        num_rel = 1
        if target[i] == indices[i]:
        #for j in range(indices.shape[1]):
            #if target[i, indices[i, j]] == 1:
            #num_rel += 1
            average_precision += float(num_rel) / count_frames
        #if num_rel==0: continue
        average_precision = average_precision / num_rel
        # print("average_precision: ", average_precision)
        map_sum += average_precision
        count_nonzero += 1
    #return map_sum / indices.shape[0]
    return float(map_sum) / count_nonzero


def precision_at_k(labels, indices, count_frames, k=1, doc_num=10):
    
    #scores = scores.reshape(-1,doc_num) # [batch, doc_num]
    #labels = labels.reshape(-1,doc_num) # [batch, doc_num]

    #sorted, indices = numpy.sort(scores, 1), numpy.argsort(-scores, 1)
    count_nonzero = 0
    precision = 0
    for i in range(len(indices)):
        #num_rel = numpy.sum(labels[i])
        #if num_rel==0: continue
        rel = 0
        for j in range(k):
            if labels[i] == indices[i]:
                rel += 1
        precision += float(rel) / float(k)
        count_nonzero += 1
    return precision / count_nonzero

In [3]:
def calculateEudDistance(i1, i2): #lower-level visual features 
    #distance = 0
    #for i in range(len(i1)):
     #   distance += np.square(i1[i]-i2[i])
    #print("i1", i1.shape)
    #print("i2", i2.shape)
    #print("i1amax", np.amax(i1))
    #print("i1amin", np.amin(i1))
    #print("i2amax", np.amax(i2))
   # print("i2amin", np.amin(i2))
    #distance = np.sqrt(np.sum((i1-i2)**2))
    distance = np.linalg.norm(i1-i2)
    #print("eud", distance)
    #return np.sqrt(distance)
    #print("norm", np.ones(( 360, 640))*256)
    #print("norm dis", np.sqrt(np.sum((np.ones(( 360,640,3))*255)**2)))
    norm_dis = np.linalg.norm(np.ones(( 360,640,3))*255)
    #print("norm_dis", norm_dis)
    distance = distance / norm_dis
    #distance = np.linalg.norm(i1-i2)/np.linalg.norm((np.ones(( 360, 640))*256-np.zeros((360, 640))))
    #print("eud2", distance)
    return distance


def analysis_accuracy(decode_list, ground_list): #DeepQAMVS: Query-Aware Hierarchical Pointer Networks for Multi-Video Summarization
    correct = 0
    distance = []
    #print("decode_list", decode_list)
    for i1, i2 in zip(decode_list, ground_list):
        i2_resize = cv2.resize(i2, (640, 360))
        i1_resize = cv2.resize(i1, (640, 360))
        #i2_norm = cv2.normalize(i2_resize, None, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32FC1)
        #i1_norm = cv2.normalize(i1_resize, None, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32FC1)
        dis = calculateEudDistance(i1_resize, i2_resize)
        distance.append(dis)
        #if calculateEudDistance(i1_norm, i2_norm) <= 0.6:
        if dis <= 0.6:
            correct += 1
    #print("correct", correct)
    #print("decode_list", decode_list)
    acc = correct/len(decode_list)
    #print("acc", acc, "distance", distance)
    return acc, distance

def get_objects(image, net):
    CONF = 0.5
    THRESHOLD = 0.5
    # load our input image and grab its spatial dimensions
    labelsPath = "YOLO\coco.names"
    LABELS = open(labelsPath).read().strip().split("\n")
    #image = cv2.imread(image)
    #(H, W) = image.shape[:2]
    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    #print("ln", ln)
    #print("net.getUnconnectedOutLayers()", net.getUnconnectedOutLayers())
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
    #ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    # show timing information on YOLO
    #print("[INFO] YOLO took {:.6f} seconds".format(end - start))
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
    # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > CONF:
                #box = detection[0:4] * np.array([W, H, W, H])
                confidences.append(float(confidence))
                classIDs.append(classID)
    #idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONF, THRESHOLD)

    detected_labels = []

    #for i in idxs.flatten():
    for i in classIDs:
        #detected_labels.append(LABELS[classIDs[i]])
        detected_labels.append(LABELS[i])

    return detected_labels, confidences

def get_iou(item1,item2):
    

    intersection = 0
    for item in item1:
        if item in item2:
            intersection+=1
    union = len(item1)+len(item2)-intersection

    if union > 0:
        return intersection*1.0 / union
    else:
        return 1

def analysis_iou(decode_list, ground_list): #Deep Reinforcement Learning for Query-Conditioned Video Summarization
    #higher-level semantic information
    # derive the paths to the YOLO weights and model configuration
    weightsPath = "YOLO\yolov3.weights"
    configPath = "YOLO\yolov3.cfg"
    # load our YOLO object detector trained on COCO dataset (80 classes)
    #print("[INFO] loading YOLO from disk...")
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    #print("net", net)

    iou_list = []
    i1_object_list = []
    i2_object_list = []
    for i1, i2 in zip(decode_list, ground_list):
        
        i1_object, i1_conf = get_objects(i1, net)
        i2_object, i2_conf = get_objects(i2, net)
        i1_object_list.append(','.join(i1_object))
        i2_object_list.append(','.join(i2_object))

        #print("i2_object", i2_object)
        # print(get_iou(i1_object,i2_object))
        iou_list.append(get_iou(i1_object,i2_object))

    return sum(iou_list)/len(iou_list), iou_list, i1_object_list, i2_object_list


def analysis_iouk(decode_list, ground_list, k): #Deep Reinforcement Learning for Query-Conditioned Video Summarization
    #higher-level semantic information
    # derive the paths to the YOLO weights and model configuration
    weightsPath = "YOLO\yolov3.weights"
    configPath = "YOLO\yolov3.cfg"
    # load our YOLO object detector trained on COCO dataset (80 classes)
    #print("[INFO] loading YOLO from disk...")
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    #print("net", net)

    iou_list = []
    i1_object_list = []
    i2_object_list = []
    for i1, i2 in zip(decode_list, ground_list):
        i1_object, i1_conf = get_objects(i1, net)
        i2_object, i2_conf = get_objects(i2, net)

        i1_obj_conf = dict(zip(i1_object, i1_conf))
        i2_obj_conf = dict(zip(i2_object, i2_conf))
        i1_obj_conf = dict(sorted(i1_obj_conf.items(), key = itemgetter(1), reverse = True)[:k]).keys()
        i2_obj_conf = dict(sorted(i2_obj_conf.items(), key = itemgetter(1), reverse = True)[:k]).keys()

        i1_object_list.append(','.join(i1_obj_conf))
        i2_object_list.append(','.join(i2_obj_conf))
       # print("i1_object", i1_object)
        #print("i2_object", i2_object)
        # print("i1_object_list", i1_object_list)
        # print("i2_object_list", i2_object_list)
        iou_list.append(get_iou(i1_object,i2_object))

    return sum(iou_list)/len(iou_list), iou_list, i1_object_list, i2_object_list

In [27]:



def main(dir):
    # dec_pos_dir = join(args.decode_dir, args.decode_pos_folder)

    # print("dec_pos_dir", dec_pos_dir)
    # dec_dir = join(args.decode_dir, args.decode_folder)
    dec_dir = dir
    print("dec_dir", dec_dir)
    #dec_dir = join(args.decode_dir, 'output')
    #with open(join(args.decode_dir, 'log.json')) as f:
    #    split = json.loads(f.read())['split']
    split = 'test'
    # ref_dir = join(args.ref_dir, split)
    ref_dir = "test"
    assert exists(ref_dir)


    # dec_pos_files = glob.glob(join(dec_pos_dir, "*.dec"))
    dec_img_files = glob.glob(join(dec_dir, "*.png"))
    file_id = [os.path.split(x)[1].replace('.png', '') for x in dec_img_files]


    decode_pos = []
    decode_pic = []
    ref_pic = []
    count_frame = []
    thumbnail = []
    video = []
    accuracy_list = []
    iou_list = []
    iou1_list = []
    iou2_list = []
    iou3_list = []
    iou4_list = []
    iou5_list = []

    ot_list = []

    counter = 0
    for i in file_id:
        # with open(join(dec_pos_dir, '{}.dec'.format(i)),'r') as f:
        #     decode_pos.append(f.read())

        decode_pic.append(cv2.imread(join(dec_dir, '{}.png'.format(i))))

        
        ref_pic.append(cv2.imread(join(ref_dir, '{}.png'.format(i[:-2]))))

        
        vidcap = cv2.VideoCapture(join(ref_dir, '{}.mp4'.format(i[:-2])))
        success,image = vidcap.read()
        print("join(ref_dir, '{}.mp4'.format(i))", join(ref_dir, '{}.mp4'.format(i[:-2])))
        print("success", success)
        print("vidcap", vidcap)
        # print("decode_pos[-1]", decode_pos[-1])
        i = 0
        count = 0
        video_frame = []
        # while success:
        #     video_frame.append(image)
        #     if i % 360:
        #         count +=1 
        #     success, image = vidcap.read()
            
        #     if count == int(decode_pos[-1].replace("tensor([", "").replace("], device='cuda:0')", "")):
        #         thumbnail.append(image)

        #     i += 1
        #     if count > 5000:
        #         break
        # #print("count", count)
        # count_frame.append(count)
        # video.append(video_frame)
        
        output, output_list = analysis_accuracy(decode_pic, ref_pic)
        accuracy_list.append(output)
        print("accuracy_list", accuracy_list)
        output, output_list, decode_object_list, ref_object_list = analysis_iou(decode_pic, ref_pic)
        iou_list.append(output)
        # print("iou_list", iou_list)
        # output, output_list, decode_object_list, ref_object_list = analysis_iouk(decode_pic, ref_pic, 1)
        # iou1_list.append(output)
        # print("iou1_list", iou1_list)

        # output, output_list, decode_object_list, ref_object_list = analysis_iouk(decode_pic, ref_pic, 2)
        # iou2_list.append(output)

        # output, output_list, decode_object_list, ref_object_list = analysis_iouk(decode_pic, ref_pic, 3)
        # iou3_list.append(output)

        # output, output_list, decode_object_list, ref_object_list = analysis_iouk(decode_pic, ref_pic, 4)
        # iou4_list.append(output)

        # output, output_list, decode_object_list, ref_object_list = analysis_iouk(decode_pic, ref_pic, 5)
        # iou5_list.append(output)

        # output = VideoCoverageLoss(torch.tensor(np.array(thumbnail)), torch.tensor(np.array(video)))
        # ot_list.append(output)
        decode_pos = []
        decode_pic = []
        ref_pic = []
        count_frame = []
        thumbnail = []
        video = []


    print("thumbnail", len(thumbnail))
    #output = MAP(decode_pos, ref_pos, count_frame)
    #metric = 'map'

    metric = 'accuracy'
    # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    #     f.write('accuracy: ' + str(sum(accuracy_list)/len(accuracy_list)))
    print(metric, str(sum(accuracy_list)/len(accuracy_list)))

    metric = 'iou'
    # # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    # #    f.write('iou: ' + str(sum(iou_list)/len(iou_list)))
    print(metric, str(sum(iou_list)/len(iou_list)))

    # metric = 'iou1'
    # # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    # #     f.write('iou1: ' + str(sum(iou1_list)/len(iou1_list)))
    # print(metric, str(sum(iou1_list)/len(iou1_list)))

    # metric = 'iou2'
    # # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    # #     f.write('iou2: ' + str(sum(iou2_list)/len(iou2_list)))
    # print(metric, str(sum(iou2_list)/len(iou2_list)))

    # metric = 'iou3'
    # # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    # #     f.write('iou3: ' + str(sum(iou3_list)/len(iou3_list)))
    # print(metric, str(sum(iou3_list)/len(iou3_list)))

    # metric = 'iou4'
    # # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    # #     f.write('iou: ' + str(sum(iou4_list)/len(iou4_list)))
    # print(metric, str(sum(iou4_list)/len(iou4_list)))

    # metric = 'iou5'
    # # with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    # #     f.write('iou5: ' + str(sum(iou5_list)/len(iou5_list)))
    # print(metric, str(sum(iou5_list)/len(iou5_list)))
          
   # metric = 'visual_ot'
    #with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    #    f.write('visual_ot: ' + str(sum(ot_list)/len(ot_list)))
    #print(metric, str(sum(ot_list)/len(ot_list)))
    #print(metric, output)
    #with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
   #     f.write(str(output))


    #output = precision_at_k(decode_pos, ref_pos, count_frame)
    #metric = 'precision_at_k'

    #print(metric, output)
    #with open(join(args.decode_dir, '{}.txt'.format(metric)), 'w') as f:
    #    f.write(str(output))

    
        




In [35]:
main("noise_fliter_topk_wd_23\\test")

dec_dir noise_fliter_topk_wd_23\test
join(ref_dir, '{}.mp4'.format(i)) test\-62Dxg2nUaw.mp4
success True
vidcap < cv2.VideoCapture 0000027EBEF0D410>
accuracy_list [1.0]
join(ref_dir, '{}.mp4'.format(i)) test\-c5fCJ6UA4M.mp4
success True
vidcap < cv2.VideoCapture 0000027EBEF0DB90>
accuracy_list [1.0, 1.0]
join(ref_dir, '{}.mp4'.format(i)) test\-csCOAvV8kY.mp4
success True
vidcap < cv2.VideoCapture 0000027EBEF0D730>
accuracy_list [1.0, 1.0, 0.0]
join(ref_dir, '{}.mp4'.format(i)) test\-mqga0a6H8I.mp4
success True
vidcap < cv2.VideoCapture 0000027EBEF0DD90>
accuracy_list [1.0, 1.0, 0.0, 0.0]
join(ref_dir, '{}.mp4'.format(i)) test\-NWKv2gLx_w.mp4
success True
vidcap < cv2.VideoCapture 0000027EBEFB0E70>
accuracy_list [1.0, 1.0, 0.0, 0.0, 0.0]
join(ref_dir, '{}.mp4'.format(i)) test\0adj6z5jk5Q.mp4
success True
vidcap < cv2.VideoCapture 0000027EBEFB0E30>
accuracy_list [1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
join(ref_dir, '{}.mp4'.format(i)) test\18jUQxhjkGc.mp4
success True
vidcap < cv2.VideoCapture 00

In [6]:
!pip install rouge

In [6]:
from rouge import Rouge

def calculate_rouge(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference, avg=True)  # avg=True计算平均值
    # return {
    #     "ROUGE-1": scores['rouge-1'],
    #     "ROUGE-2": scores['rouge-2'],
    #     "ROUGE-L": scores['rouge-l']
    # }
    return float(scores['rouge-1']['r']), float(scores['rouge-2']['r']), float(scores['rouge-l']['r'])

# 示例用法
hypothesis = "The weather is great today, the sun is shining"
reference = "The sunshine is very good today"
rouge_scores = calculate_rouge(hypothesis, reference)
print("ROUGE scores:", rouge_scores)


ROUGE scores: (0.3333333333333333, 0.0, 0.3333333333333333)


In [7]:
import json

def text_eval(dir):
    # dec_pos_dir = join(args.decode_dir, args.decode_pos_folder)

    # print("dec_pos_dir", dec_pos_dir)
    # dec_dir = join(args.decode_dir, args.decode_folder)
    dec_dir = dir
    print("dec_dir", dec_dir)
    #dec_dir = join(args.decode_dir, 'output')
    #with open(join(args.decode_dir, 'log.json')) as f:
    #    split = json.loads(f.read())['split']
    split = 'test'
    # ref_dir = join(args.ref_dir, split)
    ref_dir = "test"
    assert exists(ref_dir)


    # dec_pos_files = glob.glob(join(dec_pos_dir, "*.dec"))
    dec_img_files = glob.glob(join(dec_dir, "*.png"))
    file_id = [os.path.split(x)[1].replace('.png', '') for x in dec_img_files]


    decode_pos = []
    decode_pic = []
    ref_pic = []
    count_frame = []
    thumbnail = []
    video = []
    rough1_list = []
    rough2_list = []
    roughl_list = []
    
    for i in file_id:
        # with open(join(dec_pos_dir, '{}.dec'.format(i)),'r') as f:
        #     decode_pos.append(f.read())
        with open(join(dec_dir, '{}.txt'.format(i)), "r") as f:  # 打开文件
            summary = f.read()  # 读取文件
            # print(data)
            # decode_pic.append(data)

        
        with open(join(ref_dir, '{}.json'.format(i[:-2])), "r") as f:  # 打开文件

            data=json.load(f)
            title=data['title']
            ref_pic.append(data['title'])

    
        # print("join(ref_dir, '{}.mp4'.format(i))", join(ref_dir, '{}.mp4'.format(i[:-2])))


        i = 0
        count = 0
        video_frame = []
        
        rough1,rough2,roughl = calculate_rouge(summary, title)
        rough1_list.append(rough1)
        rough2_list.append(rough2)
        roughl_list.append(roughl)
        print("accuracy_list", rough1_list)

        decode_pos = []
        decode_pic = []
        ref_pic = []
        count_frame = []
        thumbnail = []
        video = []


    print("thumbnail", len(thumbnail))


    metric = 'accuracy'

    print(metric, str(sum(rough1_list)/len(rough1_list)))
    print(metric, str(sum(rough2_list)/len(rough2_list)))
    print(metric, str(sum(roughl_list)/len(roughl_list)))


In [30]:
text_eval("noise_filter_train_2100\\test")

dec_dir noise_filter_train_2100\test
accuracy_list [0.3333333333333333]
accuracy_list [0.3333333333333333, 0.125]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2, 0.0]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2, 0.0, 0.0]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2, 0.0, 0.0, 0.5714285714285714]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2, 0.0, 0.0, 0.5714285714285714, 0.5]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2, 0.0, 0.0, 0.5714285714285714, 0.5, 0.1]
accuracy_list [0.3333333333333333, 0.125, 0.2222222222222222, 0.0, 0.0, 0.2, 0.0, 0.0, 0.57142857142857

In [3]:
import os
from pyrouge import Rouge155

# 设置ROUGE工具包路径
os.environ['ROUGE_HOME'] = r'D:\Study\\Master_Document\\Research\\Research_Pathway\\code\\ROUGE\\pyrouge\\tools\\ROUGE-1.5.5'
os.environ['ROUGE_EVAL_HOME'] = r'D:\Study\\Master_Document\\Research\\Research_Pathway\\code\\ROUGE\\pyrouge\\tools\\ROUGE-1.5.5\data'

rouge = Rouge155()
rouge.system_dir = r'refs\\test'
rouge.model_dir = r'decoded_clipsumcomp_topic\outputText'
rouge.system_filename_pattern = '[A-z0-9_-]+.ref'
rouge.model_filename_pattern = r'([A-z0-9_-]+).dec'

# 使用Perl解释器运行ROUGE脚本
command = [
    'perl', 
    os.path.join(os.environ['ROUGE_HOME'], 'ROUGE-1.5.5.pl'), 
    '-e', os.path.join(os.environ['ROUGE_EVAL_HOME']), 
    '-c', '95', '-2', '-1', '-U', '-r', '1000', '-n', '4', '-w', '1.2', '-a', '-m',
    'C:\\Users\\Liu Sicheng\\AppData\\Local\\Temp\\tmpz8nz9n5z\\rouge_conf.xml'
]

import subprocess
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
print(result.stderr)


Exception: Cannot set data directory because the path D:\Study\Master Document\Research\Research Pathway\code\ROUGE\pyrouge\tools\ROUGE-1.5.5\data does not exist.